### converts NFDI4PSO to DPBO
the preivous NFDI4PSO terms are converted to three types of IDs:
1. DPBO:XXXXXX  
2. other ontology terms if it was in the XREF before, but now as the Term name
3. other ontology terms which is the same in NFDI4PSO and DPBO but now has a different term name.

All old term names are als converted to new (uniformed names)
  

In [3]:
import pandas as pd
import re
import numpy as np
with open("../../../nfdi4plants_ontology/dpbo.obo") as f:
    dpbo = [p.strip() for p in f.read().split('[Term]')]
with open('../../../nfdi4plants_ontology/nfdi4plants_ontology_OBSOLETE.obo', 'r') as f:
    nfdi4pso=[p.strip() for p in f.read().split("[Term]")  ]
def parse_ont(dpbo):
    dpbo_id,dpbo_name,dpbo_xref = [], [],[]
    for i in range(1, len(dpbo)):
        if dpbo[i] !="is_obsolete: true":
            id = re.search('id\: (.*)\n',dpbo[i])
            dpbo_id.append(id.group(1)) 
            name = re.search('name\: (.*)\n',dpbo[i])
            if name:
                dpbo_name.append(name.group(1))
            else:
                print("name is None: "+str(dpbo[i].split("name: ")))
                dpbo_name.append(dpbo[i].split("name: ")[1])
            xref = re.search('xref\: (.*)\n',dpbo[i])
            if xref:
                dpbo_xref.append(xref.group(1))
            else:
                dpbo_xref.append("")
        #         print("xref is None: "+dpbo[i])
    return [dpbo_id,dpbo_name,dpbo_xref]
nfdi_df = parse_ont(nfdi4pso)
dpbo_df = parse_ont(dpbo)

df1 = pd.DataFrame()
df1["dpbo_id"]  =dpbo_df[0]
df1["dpbo_name"]=dpbo_df[1]
df1["dpbo_xref"]=dpbo_df[2]

df2 = pd.DataFrame()
df2["n4p_id"]  =nfdi_df[0]
df2["n4p_name"]=nfdi_df[1]
df2["n4p_xref"]=nfdi_df[2]
df2["dpbo_id_c"]  = ["DPBO"+line.split("NFDI4PSO")[1] for line in  nfdi_df[0]]

df3 =df1.set_index('dpbo_id').join(df2.set_index('dpbo_id_c'), lsuffix='_dpbo', rsuffix='_nfdi4pso')


# terms has not a DPBOid
df4 = df2.set_index('dpbo_id_c').join(df1.set_index('dpbo_id'), lsuffix='_dpbo', rsuffix='_nfdi4pso')

df5 = df4[df4["dpbo_name"].notna()]
df_c1 = df5
df6 = df4[df4["dpbo_name"].isna()]
df6 = df6.drop(columns=["dpbo_name", "dpbo_xref"])

df7 =df6.set_index('n4p_xref').join(df1.set_index("dpbo_id"))
# the nfdi4pso xref matches the new id
df_c2=df7.dropna()
df9 = df7[df7["dpbo_name"].isna()]
df9["n4p_xref"]=df9.index
df9.index= df9.set_index("n4p_name").index.str.lower()
df9=df9.drop(columns=["dpbo_name", "dpbo_xref"])
df10 =df9.join(df1.set_index("dpbo_name"))
df10["dpbo_name"]=df10.index
df10=df10.dropna().set_index("dpbo_id")
df_c3=df10

name is None: ['id: NCIT:C126370\n', 'Cover Slip']
name is None: ['id: NCIT:C175898\n', 'Imaging Software Name']
name is None: ['id: NCIT:C181919\n', 'Objective Lens']
name is None: ['id: OBI:0000079\n', 'culture medium']
name is None: ['id: ObsoleteClass\n', 'Obsolete Class']
name is None: ['id: SCTID:71661000\n', 'cover slip preparation (procedure)']


/tmp/ipykernel_550/3656601731.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df9["n4p_xref"]=df9.index


In [4]:
df_convert = pd.concat([df_c1, df_c2, df_c3])
df_convert
df_convert["url_after"]=[s.replace(":", "_") for s in list(df_convert["n4p_id"].index)]
df_convert["url_before"]=[s.replace(":", "_") for s in list(df_convert["n4p_id"])]

In [5]:
df_convert

,n4p_id,n4p_name,n4p_xref,dpbo_name,dpbo_xref,url_after,url_before
DPBO:0000001,NFDI4PSO:0000001,growth plot design,,growth plot design,,DPBO_0000001,NFDI4PSO_0000001
DPBO:0000002,NFDI4PSO:0000002,Read Alignment Software,,read alignment software,,DPBO_0000002,NFDI4PSO_0000002
DPBO:0000003,NFDI4PSO:0000003,Read Alignment Software Version,,read alignment software version,,DPBO_0000003,NFDI4PSO_0000003
DPBO:0000004,NFDI4PSO:0000004,Read Alignment Software Parameters,,read alignment software parameters,,DPBO_0000004,NFDI4PSO_0000004
DPBO:0000005,NFDI4PSO:0000005,Humidity Day,,humidity day,,DPBO_0000005,NFDI4PSO_0000005
...,...,...,...,...,...,...,...
SIO:001051,NFDI4PSO:1000132,data analysis,NaN,data analysis,,SIO_001051,NFDI4PSO_1000132
data:2795,NFDI4PSO:1000107,ORF identifier,NaN,ORF identifier,,data_2795,NFDI4PSO_1000107
PSO:0000013,NFDI4PSO:0000071,Plant disease,,plant disease,,PSO_0000013,NFDI4PSO_0000071
GENEPIO:0001153,NFDI4PSO:0000076,Sample Collected By,NCIT:C45262,sample collected by,,GENEPIO_0001153,NFDI4PSO_0000076


In [8]:
import openpyxl
workbook = openpyxl.load_workbook('mapping_ERC000037.xlsx')
# Select the sheet you want to work on
worksheet = workbook['Sheet1']

# Loop through all the rows in the sheet and replace the string
for row in worksheet.iter_rows():
    for index, cell in enumerate(row):
        if cell.value!=None:

            s = str(cell.value)+""
            for row in df_convert.iterrows():
                cell.value = s.replace(str(row[1][0]), row[0])
                s = cell.value+""
            for row in df_convert.iterrows():
                cell.value = s.replace(row[1][1], row[1][3])
                s = cell.value+""
            for row in df_convert.iterrows():
                cell.value = s.replace(row[1][6], row[1][5])
                s = cell.value+""
# for row in worksheet.iter_rows(min_row=2):
#     row[2].value = row[1].value.split(":")[0]
# Save the updated workbook
workbook.save('example.xlsx')

In [33]:
for row in worksheet.iter_rows(min_row=2):
    print(row[2].value)



DPBO
DPBO
OBI
NCIT
EFO
EFO
EFO
OBI
EFO
DPBO
DPBO
DPBO
NCIT
PSO
DPBO
DPBO
TO
PECO
PECO
DPBO
DPBO
PECO
DPBO
DPBO
DPBO
DPBO
PECO
PECO
PECO
PECO
DPBO
GENEPIO
GENEPIO
EFO
DPBO
DPBO
DPBO
SIO
SIO
PO
DPBO
DPBO
DPBO
DPBO
ENVO
DPBO
DPBO


In [ ]:
for row in df_convert.iterrows():
    print(row[1][2])

In [ ]:
worksheet = workbook['ENA_PLANT_SAMPLE']

In [4]:
df_convert.to_csv("conversion.csv")